In [1]:
import json
import pandas as pd
import requests
import os
import numpy as np
import math
import sqlite3

from pathlib import Path
from typing import Union
from collections import defaultdict
from sqlalchemy import create_engine


In [2]:
# see website for information:
# https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/

api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# List of state codes
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
               'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
               'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# Variable to store the total number of electric vehicle stations in the US
total_stations_us = 0

# Dictionary to store the totals for each state
state_totals = {}

for state_code in state_codes:
    query_params = {
        'fuel_type_code': 'ELEC',
        'state': state_code,
        'country': 'US',
        'api_key': api_key
    }

    # Make the API request
    response = requests.get(base_url, params=query_params)

    # Parse the JSON response
    data = response.json()

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the total number of electric vehicle stations for the specified state
        total_stations = data.get('total_results', 0)

        # Increment the total stations in the US variable
        total_stations_us += total_stations

        # Store the result in the dictionary
        state_totals[state_code] = total_stations

    else:
        # Print an error message if the request was not successful
        print(f'Error: {response.status_code} - {response.text}')

# Sort the dictionary by state in decreasing order based on total stations
sorted_totals = dict(sorted(state_totals.items(), key=lambda item: item[1], reverse=True))

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the sorted totals to a JSON file in the 'data_query' folder
state_data_file_path = os.path.join(output_folder, 'electric_stations_by_state_0106.json')
with open(state_data_file_path, 'w', encoding='utf-8') as state_data_file:
    json.dump(sorted_totals, state_data_file, ensure_ascii=False, indent=4)

# Print the total number of electric vehicle stations in the US
print(f'Total electric vehicle stations in the US for fuel type ELEC: {total_stations_us}')

print(f'The sorted totals have been saved to: {state_data_file_path}')


Total electric vehicle stations in the US for fuel type ELEC: 79738
The sorted totals have been saved to: data_query/electric_stations_by_state_0106.json


In [3]:
api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# Set up parameters for the API request
params = {
    'fuel_type_code': 'ELEC',
    'country': 'US',
    'restricted_access': 'false',
    'api_key': api_key
}

# Make the API request and convert the response to a DataFrame
response = requests.get(base_url, params=params)
data = response.json()

if response.status_code == 200:
    # Extract and normalize the data
    fuel_stations_df = pd.json_normalize(data['fuel_stations'])

    # Keep only the specified fields in the DataFrame
    selected_fields = ['id', 'station_name', 'street_address', 'city', 'state', 'zip',
                       'latitude', 'longitude', 'access_code', 'ev_network', 'ev_connector_types', 'status_code',
                       'ev_pricing', 'geocode_status']
    fuel_stations_df = fuel_stations_df[selected_fields]

    # Get the total count of each unique value in the 'state' field
    total_results_by_state = fuel_stations_df['state'].value_counts()

    print(f'Total results by state:\n{total_results_by_state}')
else:
    print(f'Error: {response.status_code} - {response.text}')
    exit()


Total results by state:
state
CA    19351
NY     4391
TX     4170
FL     4055
MA     3201
WA     2557
CO     2511
GA     2298
IL     2291
PA     2185
OH     2039
NC     2024
MD     2009
MI     1917
VA     1870
MN     1565
MO     1543
AZ     1463
NJ     1438
OR     1436
TN     1139
IA     1099
UT     1025
WI     1011
IN      947
CT      932
SC      753
KS      724
OK      707
NV      685
AL      581
ME      513
AR      500
DC      442
KY      441
HI      434
NE      432
VT      414
NM      385
LA      372
RI      350
NH      295
MS      257
ID      248
DE      248
WV      214
SD      198
MT      158
ND      154
WY      142
AK       66
PR       21
KA        1
Name: count, dtype: int64


In [4]:
# API Pull for only California
api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# Selected fields to retrieve
selected_fields = ['id', 'station_name', 'street_address', 'city', 'state', 'zip',
                   'latitude', 'longitude', 'access_code', 'ev_network', 'ev_connector_types', 'status_code',
                   'ev_pricing', 'geocode_status']

# List of states to filter (Only 'CA' in this case)
state_codes = ['CA']

# Set up parameters for the API request
params = {
    'fuel_type_code': 'ELEC',
    'country': 'US',
    'restricted_access': 'false',
    'api_key': api_key
}

# List to store detailed information for stations meeting the criteria
station_data_by_state = {}

# Function to make API requests and handle errors
def make_api_request(url, params):
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json() if response.status_code == 200 else None
    except requests.exceptions.RequestException as e:
        print(f'Error making the API request: {e}')
        return None

# Make API requests for each state and filter based on state codes
for state_code in state_codes:
    params['state'] = state_code  # Include state code in the parameters

    # Make the API request
    data = make_api_request(base_url, params)
    
    if data is not None:
        # Convert the response to a DataFrame with selected fields
        stations_df = pd.json_normalize(data['fuel_stations'], sep='_')[selected_fields].fillna('N/A')
        station_data_by_state[state_code] = stations_df.to_dict(orient='records')

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the station data by state to a JSON file in the 'data_query' folder
output_file_path = os.path.join(output_folder, 'fuel_stations_selected_by_CA_0106.json')
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(station_data_by_state, output_file, ensure_ascii=False, indent=4)

print(f'The fuel station data for the state of California has been saved to: {output_file_path}')




The fuel station data for the state of California has been saved to: data_query/fuel_stations_selected_by_CA_0106.json


In [5]:
# Read in JSON data from the file
json_data_path = Path("data_query/fuel_stations_selected_by_CA_0106.json")

if json_data_path.exists():
    with json_data_path.open("r", encoding="utf8") as f:
        fuel_station_data = json.load(f)

# Check if the necessary data is present in the loaded JSON
if 'CA' in fuel_station_data:
    # Extract data for California from the loaded JSON
    ca_station_data = fuel_station_data['CA']

    # Create DataFrame
    fuel_station_df = pd.DataFrame(ca_station_data)
    
    

In [6]:
fuel_station_df.dtypes

id                      int64
station_name           object
street_address         object
city                   object
state                  object
zip                    object
latitude              float64
longitude             float64
access_code            object
ev_network             object
ev_connector_types     object
status_code            object
ev_pricing             object
geocode_status         object
dtype: object

In [7]:
# Data cleansing operations
conversion_mapping = {
    "station_name": "string",
    "street_address": "string",
    "state": "string",
    "city": "string",
    "zip": "Int64",
    "latitude": "float64",
    "longitude": "float64",
    "access_code": "string",
    "ev_network": "string",
    "ev_connector_types": "string",
    "status_code": "string",
    "ev_pricing": "string",
    "geocode_status": "string",

}

for column, dtype in conversion_mapping.items():
    try:
        if column == 'zip':
            fuel_station_df[column] = pd.to_numeric(fuel_station_df[column], errors='coerce', downcast='integer')
        elif column.startswith('ev_') and dtype == 'Int64':
            # Convert only if the value is not 'N/A'
            fuel_station_df[column] = pd.to_numeric(fuel_station_df[column].replace('N/A', pd.NA), errors='coerce', downcast='integer')
        else:
            fuel_station_df[column] = fuel_station_df[column].astype(dtype)
    except (ValueError, pd.errors.OutOfBoundsDatetime) as e:
        print(f"Error converting column {column} to {dtype}: {e}")
        # Handle the exception here, for example, assign a default value
        fuel_station_df[column] = None  # or any default value you prefer

# Display the first few rows of the DataFrame
print(fuel_station_df.head())


    id                        station_name         street_address  \
0  792  PG&E - Grass Valley Service Center     788 Taylorville Rd   
1  798    PG&E - Santa Cruz Service Center            615 7th Ave   
2  801   PG&E - Bakersfield Service Center          4101 Wible Rd   
3  806        PG&E - Fresno Service Center  3754 E California Ave   
4  809       PG&E - Salinas Service Center        390 E Alisal St   

           city state      zip   latitude   longitude access_code ev_network  \
0  Grass Valley    CA  95949.0  39.200620 -121.062126      public        N/A   
1    Santa Cruz    CA  95062.0  36.969267 -121.999295      public        N/A   
2   Bakersfield    CA  93313.0  35.317617 -119.039048      public        N/A   
3        Fresno    CA  93725.0  36.721360 -119.761165      public        N/A   
4       Salinas    CA  93901.0  36.672170 -121.641106      public        N/A   

  ev_connector_types status_code ev_pricing geocode_status  
0                N/A           E        N/A

In [8]:
fuel_station_df.dtypes

id                             int64
station_name          string[python]
street_address        string[python]
city                  string[python]
state                 string[python]
zip                          float64
latitude                     float64
longitude                    float64
access_code           string[python]
ev_network            string[python]
ev_connector_types    string[python]
status_code           string[python]
ev_pricing            string[python]
geocode_status        string[python]
dtype: object

In [9]:
 # Replace empty strings with None
fuel_station_df = fuel_station_df.replace("", None)

# Set default values for ID columns if they are null/NaN/None
id_columns = ["id", "station_name", "street_address", "city", "state", "zip",
              "latitude", "longitude", "access_code", "ev_network",
              "ev_connector_types", "status_code", "ev_pricing",
              "geocode_status"]
fuel_station_df[id_columns] = fuel_station_df[id_columns].fillna(0)

# Replace specific values if needed
fuel_station_df["ev_pricing"] = fuel_station_df["ev_pricing"].replace("FREE", "Free")

# Drop duplicate rows 
fuel_station_df = fuel_station_df.drop_duplicates()

# Display the first few rows of the DataFrame
print(fuel_station_df.head())



    id                        station_name         street_address  \
0  792  PG&E - Grass Valley Service Center     788 Taylorville Rd   
1  798    PG&E - Santa Cruz Service Center            615 7th Ave   
2  801   PG&E - Bakersfield Service Center          4101 Wible Rd   
3  806        PG&E - Fresno Service Center  3754 E California Ave   
4  809       PG&E - Salinas Service Center        390 E Alisal St   

           city state      zip   latitude   longitude access_code ev_network  \
0  Grass Valley    CA  95949.0  39.200620 -121.062126      public        N/A   
1    Santa Cruz    CA  95062.0  36.969267 -121.999295      public        N/A   
2   Bakersfield    CA  93313.0  35.317617 -119.039048      public        N/A   
3        Fresno    CA  93725.0  36.721360 -119.761165      public        N/A   
4       Salinas    CA  93901.0  36.672170 -121.641106      public        N/A   

  ev_connector_types status_code ev_pricing geocode_status  
0                N/A           E        N/A

In [10]:
import sqlite3
import pandas as pd
from pathlib import Path
import json

# Read in JSON data from the file
json_data_path = Path("data_query/fuel_stations_selected_by_CA_0106.json")

if json_data_path.exists():
    with json_data_path.open("r", encoding="utf8") as f:
        fuel_station_data = json.load(f)

    # Check if the necessary data is present in the loaded JSON
    if 'CA' in fuel_station_data:
        # Extract data for California from the loaded JSON
        ca_station_data = fuel_station_data['CA']

        # Create DataFrame
        fuel_station_df = pd.DataFrame(ca_station_data)

        # Clean the DataFrame
        fuel_station_df = fuel_station_df.replace("", None)
        id_columns = ["id", "station_name", "street_address", "city", "state", "zip",
                      "latitude", "longitude", "access_code", "ev_network",
                      "ev_connector_types", "status_code", "ev_pricing",
                      "geocode_status"]
        fuel_station_df[id_columns] = fuel_station_df[id_columns].fillna(0)
        fuel_station_df["ev_pricing"] = fuel_station_df["ev_pricing"].replace("FREE", "Free")

        # Filter DataFrame to columns with lists
        columns_with_lists = fuel_station_df.select_dtypes(include='object').columns[
            fuel_station_df.select_dtypes(include='object').applymap(lambda x: isinstance(x, list)).any()
        ]
        fuel_station_df = fuel_station_df.drop(columns=columns_with_lists)

        # Drop duplicate rows
        fuel_station_df = fuel_station_df.drop_duplicates()

        # Define the path to your SQLite database file
        database_path = Path("ev_stations.db")

        # Connect to the SQLite database
        conn = sqlite3.connect(database_path)

        # Create a cursor object to execute SQL queries
        cursor = conn.cursor()

        # Define the SQL statement to create a table
        create_table_query = """
        CREATE TABLE IF NOT EXISTS fuel_stations (
            id INTEGER PRIMARY KEY,
            station_name TEXT,
            street_address TEXT,
            city TEXT,
            state TEXT,
            zip INTEGER,
            latitude REAL,
            longitude REAL,
            access_code TEXT,
            ev_network TEXT,
            ev_connector_types TEXT,
            status_code TEXT,
            ev_pricing TEXT,
            geocode_status TEXT
        );
        """

        # Execute the SQL query to create the table
        cursor.execute(create_table_query)

        # Replace 'N/A' with None for SQLite compatibility
        fuel_station_df_cleaned = fuel_station_df.replace({'N/A': None, pd.NA: None})

        # Explicitly set data types for problematic columns
        numeric_columns = ['zip', 'latitude', 'longitude']
        for column in numeric_columns:
            fuel_station_df_cleaned[column] = pd.to_numeric(fuel_station_df_cleaned[column], errors='coerce', downcast='integer')

        # Convert DataFrame to SQLite table without specifying data types
        fuel_station_df_cleaned.to_sql("ev_stations", conn, if_exists="replace", index=False)

        # Commit the changes and close the connection
        conn.commit()
        conn.close()

        print(f"The SQLite database has been created, and the DataFrame has been stored in the 'fuel_stations' table.")
    else:
        print(f"No data available for California in the loaded JSON.")
else:
    print(f"JSON data file not found.")


The SQLite database has been created, and the DataFrame has been stored in the 'fuel_stations' table.
